In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapezoid

: 

# Fourier Series Analysis

## Core functions

In [ ]:
def compute_Fourier_Series(x, t, K, T=None, t0=None):
    """
    Compute Fourier series coefficients (exponential form) for k in [-K, ..., K].

    Inputs:
        x : numpy array (N,)  -- sampled signal (real or complex)
        t : numpy array (N,)  -- time instants (uniformly spaced)
        K : int               -- highest harmonic index (non-negative)
        T : float or None     -- fundamental period. If None, inferred from t as T = t[-1]-t[0]+dt
        t0: float or None     -- integration start. If None, uses t[0]

    Returns:
        c  : numpy array (2K+1,) complex  -- coefficients corresponding to ks
        ks : numpy array (2K+1,) int      -- harmonic indices (from -K to K)
    """
    x = np.asarray(x)
    t = np.asarray(t)
    if x.shape != t.shape or t.ndim != 1:
        raise ValueError("x and t must be 1D arrays of same shape")
    dt = t[1] - t[0]
    if T is None:
        T = (t[-1] - t[0]) + dt
    if t0 is None:
        t0 = t[0]
    ks = np.arange(-K, K+1)
    c = np.zeros(len(ks), dtype=np.complex128)
    for i, k in enumerate(ks):
        integrand = x * np.exp(-1j * 2.0 * np.pi * k * (t - t0) / T)
        c[i] = (1.0 / T) * trapezoid(integrand, t)
    return c, ks

def reconstruct_from_coeffs(c, ks, t, T=None, t0=None):
    """
    Reconstruct signal from coefficients c and harmonic indices ks at times t.
    """
    t = np.asarray(t)
    dt = t[1] - t[0]
    if T is None:
        T = (t[-1] - t[0]) + dt
    if t0 is None:
        t0 = t[0]
    x_hat = np.zeros_like(t, dtype=np.complex128)
    for ci, k in zip(c, ks):
        x_hat += ci * np.exp(1j * 2.0 * np.pi * k * (t - t0) / T)
    return x_hat

# Utility: continuous L2 error (sqrt integral |err|^2 dt)
def l2_error_continuous(x, x_hat, t):
    return np.sqrt(trapezoid(np.abs(x - x_hat)**2, t))

: 

## a) Exploring Polynomial Structures with Fourier Series (Chebyshev Polynomials)

In [ ]:
def part_a_chebyshev_T5(dt=0.001, T=2.0, K_max=60):
    """
    (a) Model x(t) = T5(tau) with fundamental period T=2s.
    Tau maps t in [0,2) -> [-1,1] with tau = 2*(t/T) - 1 -> tau = t - 1 when T=2.
    Produces:
      - error vs K
      - overlay of original and recon for small & large K
      - coefficient spectrum for a large K
    """
    t = np.arange(0, T, dt)
    tau = 2.0 * (t / T) - 1.0  # mapping to [-1,1], but for T=2 simplifies to t-1
    # Chebyshev T5(x) = cos(5 * arccos(x)) for x in [-1,1]
    x = np.cos(5.0 * np.arccos(tau))

    Ks = np.arange(1, K_max+1)
    errors = []
    for K in Ks:
        c, ks = compute_Fourier_Series(x, t, K=K, T=T, t0=t[0])
        xhat = reconstruct_from_coeffs(c, ks, t, T=T, t0=t[0]).real
        errors.append(l2_error_continuous(x, xhat, t))

    # Pick small and large K for overlay:
    K_small = max(3, min(5, K_max//10))
    K_large = K_max

    c_small, ks_small = compute_Fourier_Series(x, t, K=K_small, T=T, t0=t[0])
    xhat_small = reconstruct_from_coeffs(c_small, ks_small, t, T=T, t0=t[0]).real

    c_large, ks_large = compute_Fourier_Series(x, t, K=K_large, T=T, t0=t[0])
    xhat_large = reconstruct_from_coeffs(c_large, ks_large, t, T=T, t0=t[0]).real

    # Plotting
    fig, axs = plt.subplots(3, 1, figsize=(10, 12))
    axs[0].plot(Ks, errors, '-o', markersize=4)
    axs[0].set_xlabel('K (highest harmonic index)')
    axs[0].set_ylabel('L2 approximation error')
    axs[0].set_title('Approximation error ||x(t) - x̂(t)|| vs K')

    axs[1].plot(t, x, label='Original x(t) = T5(tau)', linewidth=1.2)
    axs[1].plot(t, xhat_small, '--', label=f'Reconstruction K={K_small}')
    axs[1].plot(t, xhat_large, ':', label=f'Reconstruction K={K_large}')
    axs[1].set_xlim(0, 2.0)
    axs[1].set_title('Original vs Fourier reconstructions (small vs large K)')
    axs[1].legend()

    # Spectrum for large K
    axs[2].stem(ks_large, np.abs(c_large), basefmt=" ")
    axs[2].set_xlabel('k (harmonic index)')
    axs[2].set_ylabel('|c[k]|')
    axs[2].set_title(f'Fourier coefficients magnitude |c[k]| for K={K_large}')
    plt.tight_layout()
    plt.show()

    return {"t": t, "x": x, "Ks": Ks, "errors": np.array(errors),
            "c_large": c_large, "ks_large": ks_large}


In [ ]:
print("(a) Chebyshev T5 demo...")
out_a = part_a_chebyshev_T5(dt=0.001, T=2.0, K_max=60)

## b) The Mystery of Gibbs Phenomenon

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numpy as np

def periodic_rect(t, p=1.0, T=1.0, t0=0.0):
    """
    Periodic rectangular pulse (square wave).

    For p=1 and T=1:
        f(t) = 1 for 0 <= (t - t0) mod T < 0.5
        f(t) = 0 for 0.5 <= (t - t0) mod T < 1

    Parameters
    ----------
    t : array_like
        Time values.
    p : float
        Pulse width (default 1.0, but here we use it just as a flag).
    T : float
        Period (default 1.0).
    t0 : float
        Time shift (default 0.0).
    """
    # Shift into [0, T)
    t_shifted = (t - t0) % T
    return np.where(t_shifted < (T/2), 1.0, 0.0)



def part_b_gibbs(dt=0.001, T=1.0, p=None, K_list=[3, 9, 30]):
    """
    Demonstrate Gibbs phenomenon on a periodic rectangular pulse.

    Default case:
    - Assignment states p=1, T=1 (full period = 1).
    - We shift by t0 = T/2 so that the pulse is 1 on [0.5, 1) and 0 on [0, 0.5),
      creating a discontinuity at t=0.5 where Gibbs phenomenon is visible.
    """
    if p is None:
        p = T   # full-period pulse width
    t0 = T / 2  # shift to create visible discontinuity

    # Time grid
    t = np.arange(0, T, dt)
    x = periodic_rect(t, p=p, T=T, t0=t0)

    # Fourier reconstructions
    reconstructions = {}
    for K in K_list:
        c, ks = compute_Fourier_Series(x, t, K=K, T=T, t0=0.0)
        xhat = reconstruct_from_coeffs(c, ks, t, T=T, t0=0.0).real
        reconstructions[K] = (xhat, c, ks)

    # Plot full period
    fig = plt.figure(figsize=(11, 7))
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.plot(t, x, label='Original rectangle', linewidth=1)
    for K in K_list:
        ax1.plot(t, reconstructions[K][0], label=f'K={K}', alpha=0.9)
    ax1.set_title('Rectangular pulse and Fourier approximations (full period)')
    ax1.set_xlim(0, T)
    ax1.legend()


    # Zoom near discontinuity at t=0.5
    ax2 = fig.add_subplot(2, 1, 2)
    zoom_win = 0.1  # half-width of zoom window
    center = 0.5

    # Pick region around t=0.5
    mask = (t >= center - zoom_win) & (t <= center + zoom_win)
    t_zoom = t[mask]
    x_zoom = x[mask]

    ax2.plot(t_zoom, x_zoom, label='Original', linewidth=1)
    for K in K_list:
        xhat_full = reconstructions[K][0]
        ax2.plot(t_zoom, xhat_full[mask], label=f'K={K}')

    ax2.set_title('Zoom near discontinuity at t=0.5 (showing Gibbs overshoot)')
    ax2.set_xlabel('time (s)')
    ax2.legend()
    plt.tight_layout()
    plt.show()


    # Overshoot estimate around t=0.5
    Kmax = max(K_list)
    xhat_max, _, _ = reconstructions[Kmax]

    center = 0.5
    win = 0.1  # look ±0.1 around the discontinuity
    idxs = np.where((t >= center - win) & (t <= center + win))[0]

    overshoot_value = np.max(xhat_max[idxs]) - 1.0


    return {
        "t": t,
        "x": x,
        "reconstructions": reconstructions,
        "overshoot_estimate": overshoot_value,
    }



In [ ]:
print("(b) Gibbs phenomenon demo...")
out_b = part_b_gibbs(dt=0.001, T=1.0, p=1, K_list=[3,9,30])
print("Estimated overshoot (largest K):", out_b["overshoot_estimate"])

Q) Why does the Fourier approximation overshoot near discontinuities?

The Fourier series overshoots near discontinuities because smooth sinusoids cannot instantaneously follow a jump, so they oscillate around it. Even as we add more harmonics, the oscillations compress but the peak overshoot remains, since the series converges in energy but not uniformly at the discontinuity. This is why the Gibbs phenomenon is a persistent feature of Fourier approximations to discontinuous signals.

Q) Why does this effect not vanish as K→∞?


Increasing
K (more harmonics) makes the approximation closer everywhere except at the discontinuity. The overshoot becomes narrower (confined to a smaller region around the jump), but its height does not decrease.

In fact, the overshoot settles to a fixed fraction (~9%) of the jump size, no matter how large K is.

## c) Real-World Connection: Distorted Audio Signals (Clipped Sine wave)

In [ ]:
def part_c_clipped_sinusoid(dt=0.001, T=1.0, f0=5.0, clip_thresh=0.7, K_values=[3,5,30]):
    """
    Clipped sinusoid x(t):
        x(t) =  1   if sin(2π f0 t) >  clip_thresh
              = -1   if sin(2π f0 t) < -clip_thresh
              = sin(2π f0 t) otherwise
    Period T = 1 (since f0 = 5 Hz → period 0.2 s, which divides 1 exactly).

    Uses consistent T and t0 in both coefficient computation and reconstruction.
    """
    # time grid
    t = np.arange(0, T, dt)

    # unclipped sinusoid
    s = np.sin(2.0 * np.pi * f0 * t)

    # apply clipping
    x = np.where(s > clip_thresh, 1.0,
                 np.where(s < -clip_thresh, -1.0, s))

    results = {}
    for K in K_values:
        # compute Fourier coefficients with consistent T and t0
        c, ks = compute_Fourier_Series(x, t, K=K, T=T, t0=0.0)

        # reconstruct with same T and t0
        xhat = reconstruct_from_coeffs(c, ks, t, T=T, t0=0.0).real

        results[K] = {"c": c, "ks": ks, "xhat": xhat}

    # plot original and reconstructions
    plt.figure(figsize=(10,6))
    plt.plot(t, x, label='Clipped signal (original)', linewidth=1.2)
    for K in K_values:
        plt.plot(t, results[K]["xhat"], label=f'Recon K={K}')
    plt.xlim(0, 1.0)
    plt.title('Clipped sinusoid and Fourier reconstructions')
    plt.xlabel('time (s)')
    plt.legend()
    plt.show()

    # spectrum for larger K
    Kplot = max(K_values)
    plt.figure(figsize=(8,4))
    plt.stem(results[Kplot]["ks"], np.abs(results[Kplot]["c"]), basefmt=" ")
    plt.xlabel('k (harmonic index)')
    plt.ylabel('|c[k]|')
    plt.title(f'Spectrum |c[k]| for clipped sinusoid (K={Kplot})')
    plt.show()

    return {"t": t, "x": x, "results": results}


In [ ]:
print("part (c) Clipped sinusoid demo...")
out_c = part_c_clipped_sinusoid(dt=0.001, T=1.0, f0=5.0, clip_thresh=0.7, K_values=[3,5,30])